In [ ]:
import numpy as np
import pandas as pd
import pingouin as pg
from sklearn import linear_model

In [ ]:
pd.set_option("display.max_rows", None, "display.max_columns", None, "display.width", 1000)

Pulire df di Prolific

In [ ]:
df1 = pd.read_csv('Dati_prolific.csv')

#drop columns

df1 = df1.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'RecordedDate',
                      'Duration (in seconds)', 'Finished', 'RecipientLastName',
                      'RecipientFirstName', 'RecipientEmail', 'ExternalReference',
                      'LocationLatitude', 'LocationLongitude', 'DistributionChannel',
                      'UserLanguage', 'ProlificID', 'PROLIFIC_PID'
                      ])
df1 = df1.drop([0, 1], axis=0)

Colonne Prolific

 DATI GRATIS

In [ ]:
df2 = pd.read_csv('Dati_gratis.csv')
# Colonne gratis
df2 = df2.drop(columns=['StartDate', 'EndDate', 'Status', 'IPAddress', 'Progress', 'RecordedDate',
                      'Duration (in seconds)', 'Finished', 'RecipientLastName',
                      'RecipientFirstName', 'RecipientEmail', 'ExternalReference',
                      'LocationLatitude', 'LocationLongitude', 'DistributionChannel',
                      'UserLanguage'
                      ])
df2 = df2.drop([0, 1], axis=0)
# Dataframe finale
df = pd.concat([df1, df2])

In [ ]:
#togliamo i clicks

for col in df.columns:
    if "Click" in col:
        del df[col]

In [ ]:
df = df.drop(columns = ['Social_9_TEXT', 'Altraroba', 'Commenti'])

In [ ]:
#togliamo i nan
df = df.dropna(axis = 0)
df.head()
df.shape[0]

In [ ]:
#save data
df.to_csv('Dati.csv')

 RIORGANIZZARE IL DATAFRAME

In [ ]:
df = pd.read_csv('Dati.csv')

Cancellare tutto dopo il trattino " _ "

In [ ]:
df.columns = df.columns.str.split('_').str[0]

Questionari #<br>
Trasformare in numerico

In [ ]:
cols = ['PU1', 'PU2', 'PU3', 'OverU1', 'OverU2', 'OverU3', 'AGRR', 'AGR', 'COS', 'COSR', 'STAB', 'STABR',
        'EX', 'EXR', 'OP', 'OPR']
df[cols] = df[cols].apply(pd.to_numeric, errors = 'coerce', axis = 1)

Items al contrario

In [ ]:
df['AGR2'] = 100 - df['AGRR']
df['COS2'] = 100 - df['COSR']
df['STAB2'] = 100 - df['STABR']
df['EX2'] = 100 - df['EXR']
df['OP2'] = 100 - df['OPR']

Calcolo dei punteggi<br>
BIG 5

In [ ]:
df['Agreeableness'] = (df['AGR'] + df['AGR2'])/2
df['Conscientiousness'] = (df['COS'] + df['COS2'])/2
df['Em Stability'] = (df['STAB'] + df['STAB2'])/2
df['Extroversion'] = (df['EX'] + df['EX2'])/2
df['Openness'] = (df['OP'] + df['OP2'])/2

Social media use

In [ ]:
df['Social Use'] = (df['PU1'] + df['PU2'] + df['PU3'] + df['OverU1'] + df['OverU2'] + df['OverU3'])/6
df = df.drop(columns=['PU1', 'PU2', 'PU3', 'OverU1', 'OverU2', 'OverU3', 'AGRR', 'AGR', 'COS', 'COSR', 'STAB', 'STABR',
                      'EX', 'EXR', 'OP', 'OPR', 'AGR2', 'COS2', 'STAB2', 'STAB2', 'EX2', 'OP2'])

In [ ]:
df = df.drop(columns=['Unnamed: 0'])

CONTROLLARE ALPHA DI CRONBACH PER SOCIAL MEDIA USE

Ricordati di riportarla da qualche parte

In [ ]:
#cronb = df[['PU1','PU2','PU3','OverU1','OverU2','OverU3']]
#g.cronbach_alpha(data = cronb, ci = .99)

Formattare diversamente il dataframe

In [ ]:
longdf = pd.melt(frame=df, id_vars=['ResponseId', 'Genere', 'Età', 'Campo', 'Studio', 'Social', 'dx/sx',
                                    'Cons/prog', 'Ind/col', 'Tempo', 'Flame', 'Agreeableness',
                                    'Conscientiousness', 'Em Stability', 'Extroversion',
                                    'Openness', 'Social Use'], var_name="Domanda", value_name="Risposta")

In [ ]:
longdf.to_csv('Lungo_df_pulito.csv')

In [ ]:
longdf['Topic'] = longdf['Domanda'].apply(lambda x: x.split('.')[0])
longdf['Codicino'] = longdf['Domanda'].apply(lambda x: x.split('.')[1])

Delete the column that had both

In [ ]:
longdf = longdf.drop(columns='Domanda')

In [ ]:
result = longdf.pivot(index=['ResponseId', 'Genere', 'Età', 'Campo', 'Studio', 'Social', 'dx/sx',
                                    'Cons/prog', 'Ind/col', 'Tempo', 'Flame',
                                    'Agreeableness', 'Conscientiousness', 'Em Stability', 'Extroversion',
                                    'Openness', 'Social Use', 'Topic'], columns='Codicino', values='Risposta')

In [ ]:
result = result.reset_index()
result.to_csv('Dati_puliti.csv')

In [ ]:
df = pd.read_csv('Dati_puliti.csv')
#df2 = df[["ALDIS", "ALEM", "IODIS", "IOEM"]]
#df2.corr()
#pearsonr(df2.IOEM, df2.ALEM)

# Ora estraiamo i predittori che sicuro ci serviranno

In [ ]:
predittori = df.loc[df['Topic'].str.contains('EUT')]
predittori = predittori[['ResponseId', 'dx/sx', 'Cons/prog', 'Ind/col', 'Flame', 'Agreeableness',
                     'Conscientiousness', 'Em Stability', 'Extroversion', 'Openness',
                     'Social Use']]

In [ ]:
predittori = predittori.sort_values(by=['ResponseId'], ascending=False)
predittori.head()

In [ ]:
predittori.to_csv('Predittori.csv')

# PROIEZIONE SOCIALE: IL CODICE DI ROB

togliere colonne che non servono

In [ ]:
df.head()
df2 = df[["ResponseId", "Topic", "ALDIS", "ALEM", "IODIS", "IOEM"]]

Separate dataframes for each subject

In [ ]:
lista_dicts = []
for name in set(df2['ResponseId']):
    temp_df = df2.loc[df2['ResponseId'] == name]
    temp_df = temp_df.drop(columns=['ResponseId'])
    dict_df = {'Name': name, 'Df': temp_df}
    lista_dicts.append(dict_df)

In [ ]:
core = pd.DataFrame(columns=['Name', 'Corr'])

Append columns for each dataframe

In [ ]:
for subject in lista_dicts:
    sub_df = subject.get("Df")
    s1a = sub_df['ALDIS']
    s1b = sub_df['ALEM']
    s2a = sub_df['IODIS']
    s2b = sub_df['IOEM']
    df0 = pd.DataFrame({'A': pd.concat([s1a, s1b]), 'I': pd.concat([s2a, s2b])})
    subject["Df"] = df0
    subject["Corr"] = np.corrcoef(df0['A'], df0['I'])[0, 1]
    core.loc[len(core.index)] = [subject.get("Name"), subject.get("Corr")]

In [ ]:
core = core.sort_values(by=['Name'], ascending=False)
core.to_csv('core.csv')

In [ ]:
#check dimensioni
core.shape[0]
predittori.shape[0]

 U N I R E

In [ ]:
core = pd.read_csv('core.csv')
predittori = pd.read_csv('Predittori.csv')

In [ ]:
lasso_df = pd.concat([core, predittori], axis=1)
lasso_df.head()

In [ ]:
lasso_df = lasso_df.drop(columns = ['ResponseId', 'Unnamed: 0'])

In [ ]:
lasso_df.to_csv('DatixAnalisi.csv')

In [ ]:
lasso_df = pd.read_csv('DatixAnalisi.csv')
lasso_df.head()